In [ ]:
import Pkg

Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Flux")
Pkg.add("MLDataUtils")

In [2]:
using Random

using CSV
using DataFrames
using Flux
using Flux: onehotbatch
using MLDataUtils
using Statistics: mean

## Loading and Cleaning

In [4]:
# Load the dataset
folder_path = joinpath(@__DIR__, "../dat/Churn_Modelling.csv")
data = CSV.File(folder_path) |> DataFrame

println(size(data))

(10000, 14)


In [5]:
# Drop irrelevant columns
select!(data, Not([:RowNumber, :CustomerId, :Surname]));

In [6]:
# Handle categorical variable 'Geography'
# Extract unique values for one-hot encoding
geography_values = unique(data.Geography)
println("Unique Geography Values: ", geography_values)

# Perform one-hot encoding for 'Geography'
for value in geography_values
    data[:, Symbol(value)] = (data.Geography .== value) .|> Int
end
select!(data, Not(:Geography));

Unique Geography Values: String7["France", "Spain", "Germany"]


In [7]:
# Encode 'Gender' as binary (Female=0, Male=1)
data.Gender = (data.Gender .== "Male") .|> Int;

In [8]:
# Normalize numerical features
numeric_columns = [:CreditScore, :Age, :Balance, :EstimatedSalary]
for col in numeric_columns
    col_min = minimum(data[!, col])
    col_max = maximum(data[!, col])
    data[!, col] = (data[!, col] .- col_min) ./ (col_max - col_min)
end

## Train Test Split

In [9]:
function train_test_split(df::DataFrame, ratio::Float64, target_col::Symbol, seed::Real; balanced::Bool = false)
    # Ensure that the ratio is valid
    if ratio <= 0 || ratio >= 1
        error("Ratio must be between 0 and 1.")
    end

    # Set the random seed for reproducibility
    Random.seed!(seed)

    # Group the data by the `default` column
    grouped = groupby(df, target_col)

    # Initialize empty DataFrames for train and test sets
    train = DataFrame()
    test = DataFrame()

    
   # If balancing, find the minimum group size
   max_size = Inf64
   if balanced
       max_size = minimum(nrow(g) for g in grouped)
   end

    # Iterate through each group and perform the stratified split
    for g in grouped
        n = nrow(g)

        # Adjust the train size based on balancing
        if balanced
            n = max_size
        end

        train_size = Int(round(ratio * n))
        indices = shuffle(1:n)

        train_indices = indices[1:train_size]
        test_indices = indices[train_size+1:end]

        train = vcat(train, g[train_indices, :])
        test = vcat(test, g[test_indices, :])
    end

    return train, test
end;

In [10]:
# Count occurrences of each unique value in the 'Exited' column
counts = combine(groupby(data, :Exited), nrow => :Count)

# Display the counts
println(counts)

2×2 DataFrame
 Row │ Exited  Count 
     │ Int64   Int64 
─────┼───────────────
   1 │      0   7963
   2 │      1   2037


In [11]:
# Perform train-test split
train_data, test_data = train_test_split(data, 0.8, :Exited, 420, balanced=false)

# Convert the target column to a vector for compatibility
y_train = Float64.(train_data.Exited)
X_train = Matrix(select(train_data, Not(:Exited)))

y_test = Float64.(test_data.Exited)
X_test = Matrix(select(test_data, Not(:Exited)));

println("Train X: $(size(X_train))") 
println("Train Y: $(length(y_train))")  
println("Test X:  $(size(X_test))") 
println("Test Y:  $(length(y_test))")  


Train X: (8000, 12)
Train Y: 8000
Test X:  (2000, 12)
Test Y:  2000


In [12]:
# Define the Flux model for logistic regression
model = Chain(
    Dense(12, 2),
    softmax
)

Chain(
  Dense(12 => 2),                       # 26 parameters
  NNlib.softmax,
) 

In [13]:
# Define the classes for one-hot encoding
const classes = unique(y_train)
flux_y_train_onehot = onehotbatch(y_train, classes)
flux_y_test_onehot = onehotbatch(y_test, classes);

In [15]:
# Define the loss function
function flux_loss(flux_model, features, labels_onehot)
    ŷ = flux_model(features)
    Flux.logitcrossentropy(ŷ, labels_onehot)
end;

In [16]:
function train_flux_model!(f_loss, model, features, labels_onehot; lr=0.01)
    dLdm, _, _ = gradient(f_loss, model, features, labels_onehot)
    @. model[1].weight = model[1].weight - lr * dLdm[:layers][1][:weight]
    @. model[1].bias = model[1].bias - lr * dLdm[:layers][1][:bias]
end;

In [17]:
# Prepare data for training
X_train_t = permutedims(Float32.(X_train)) 
println("Training X set: $(size(X_train_t))")
println("Training Y set: $(size(flux_y_train_onehot))")

Training X set: (12, 8000)
Training Y set: (2, 8000)


In [18]:
function accuracy_flux(x,y, model)
    return mean(Flux.onecold(model(x), classes) .== y)
end;

In [19]:
# Parameters
lr = 1           # Initial learning rate
final_lr = 1e-4  # Final learning rate
epochs = 15000    # Maximum number of epochs
target_accuracy = 0.95  # Desired accuracy threshold


# Training loop
for epoch = 1:epochs
    train_flux_model!(flux_loss, model, X_train_t, flux_y_train_onehot, lr=lr)
    
    # Update the learning rate using linear decay
    lr = lr - (1 - final_lr) / epochs

    # Evaluate accuracy on the training set (or validation set)
    acc = accuracy_flux(X_train_t, y_train, model)

    # Print progress
    if epoch % 1000 == 0 || acc >= target_accuracy
        println("Epoch: ", epoch, " Loss: ", flux_loss(model, X_train_t, flux_y_train_onehot))
        println("       Acc: $(acc)")
    end

    # Stop if accuracy exceeds the target
    if acc >= target_accuracy
        println("Stopping early: Target accuracy of $target_accuracy achieved.")
        break
    end
end


Epoch: 1000 Loss: 0.5168992
       Acc: 0.79625
Epoch: 2000 Loss: 0.5153429
       Acc: 0.79625
Epoch: 3000 Loss: 0.5009886
       Acc: 0.80925
Epoch: 4000 Loss: 0.49151933
       Acc: 0.819625
Epoch: 5000 Loss: 0.487897
       Acc: 0.826
Epoch: 6000 Loss: 0.48608282
       Acc: 0.828125
Epoch: 7000 Loss: 0.48490274
       Acc: 0.829
Epoch: 8000 Loss: 0.4840348
       Acc: 0.8295
Epoch: 9000 Loss: 0.4833677
       Acc: 0.8295
Epoch: 10000 Loss: 0.48284987
       Acc: 0.830375
Epoch: 11000 Loss: 0.48245186
       Acc: 0.83075
Epoch: 12000 Loss: 0.48215598
       Acc: 0.830875
Epoch: 13000 Loss: 0.48195115
       Acc: 0.830875
Epoch: 14000 Loss: 0.48183063
       Acc: 0.83125
Epoch: 15000 Loss: 0.48179078
       Acc: 0.83125


## Testing

In [21]:
X_test_t = permutedims(Float32.(X_test))
println("Test Accuracy: $(accuracy_flux(X_test_t, y_test, model))")

Test Accuracy: 0.828
